In [ ]:
import numpy as np
import pandas as pd
#numpy是一个矩阵运算库
#pandas是一个更高级的矩阵运算库，可以处理缺失，重复，异常的数值


In [ ]:
filepath='D:\\Stock prediction\\jupyter\\data\\rlData.csv'
# 这里的路径是自己的,一定要注意文件格式，要加“   \\   ”
data=pd.read_csv(filepath)
#用read_csv函数来读取文件的同时将文件保存在一个data变量中
#这之后data就是一个DataFrame的变量了（一个二维表格数据结构）
#DataFrame对象对数据进行各种操作，如数据清洗、处理、分析和可视化等。
data=data.sort_values('Date')
#这是一个按日期来排序的函数，data现在已经按日期拍好序列了
data.head()
#这个函数默认能够查看对象（dataframe）表格的前五行数据，可以通过传入int来表示查看前几行数据
data.shape
#可以查看表的维度

In [ ]:
import matplotlib.pyplot as plt
#绘制图形
import seaborn as sns
#数据可视化的更高级的库
sns.set_style("darkgrid")
#调用一种深色风格的图表，这在seaborn中是已经预先定义好的
plt.figure(figsize = (15,9))
#创建一个指定大小的窗口
#问题：我如何确定这两者是作用在同一个窗口上的呢？
plt.plot(data[['Close']])
#从这里来看sns.setstyle好像没用
plt.xticks(range(0,data.shape[0],20),data['Date'].loc[::20],rotation=45)
#把横坐标换成了日期
#range函数设置了间隔，
# data['Date'].loc[::20]的结果是一个包含原始Date列中每隔20个位置的值的子集
#其原因在于横坐标不能够太过于稠密
plt.title("****** stock price",fontsize=18,fontweight='bold')
#三个参量，一个标题，一个大小，一个加粗
plt.xlabel('Date',fontsize=18)
plt.ylabel('close price',fontsize=18)
plt.show()      


  到这里而言我们实现了股票数据（csv格式）的可视化


  下一步就是要提取出有价值的信息


In [ ]:
price=data[['Close','Low']]
price.head(6)
##price.info
##输出信息


归一化操作


In [ ]:
#归一化操作（在py中直接封装成函数了）
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(-1,1))
#MinMaxScaler是一个库函数，其内有归一化处理的操作，可以换范围
price['Close']=scaler.fit_transform(price['Close'].values.reshape(-1,1))
#对Close进行归一化
price['Low']=scaler.fit_transform(price['Low'].values.reshape(-1,1))
#对Low进行归一化操作
price
##你现在对数据进行缩放，显示结果后要反缩放回去


制作数据集

In [ ]:
def split_data(stock,lookback):
    #把数据集划分为训练集和测试集合
    ##lookback是一个指定的训练集合长度
    data_raw=stock.to_numpy()
    #stock在这里是不是一个dataframe对象呀？
    #这里是调用了“to_numpy”函数，表示能够将stock数据转化为numpy数组
    data=[]
    #创建一个空链表，用来存储切片后的数据
    #data在这里就是一个表格，
    #py中的切片操作，没有传入是默认开始于0，结束于0，步长为1
    for index in range(len(data_raw)-lookback):
        data.append(data_raw[index:index+lookback])
        #有一个py中常用的切片操作
        #把已经划分好的股票数据加入到data中去
    data=np.array(data)
    #将data转换为numpy类型
    test_set_size=int (np.round(0.2*data.shape[0]))
    ##np.round是一个四舍五入的函数,也可以使用(int)来进行转换，不过数据就只是向下取整了
    ##当然，你可以换一种方式来提高效率比如(int)(0.2*data.shape[0]+0.5)
    #data.shape[0]返回的是数组的第一个维度的大小
    train_set_size =data.shape[0]-(test_set_size)
    #训练集的大小
    x_train=data[:train_set_size,:-1,:]#这是训练特征集
    #假设为（深度，高度，宽度）
    #这里有三个维度(","划分出来)，且听我细细道来：假设有一百个元素
    #1：从0到80就是训练集     深度的前80
    #2：从“0”所指代的元素(实际上的大小为步长，从[0,0]到[0,1],留下了最后一个[0,2])
        #每一个高度的除最后一个的所有元素
    #3：宽度的所有
    y_train=data[:train_set_size,:-1,:]#这是训练标签集
    x_test=data[train_set_size:,:-1]#测试特征集
    y_test=data[train_set_size:,-1,:]#测试标签集
    
    return [x_train,y_train,x_test,y_test]
  ##叮！ https://yiyan.baidu.com/share/DqLQASp1VH 
    

In [ ]:
lookback = 20
#这里是对数据集中的一个最小单位来指定的步长进行操作
x_train, y_train, x_test, y_test = split_data(price, lookback)
#调用了上方的一个函数
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

##不对！！！！！！最后的一个

##如果加上low的变量，最后一点就是2


3.模型的构建

我们已经完成了数据的可视化（简单版），对要处理的数据进行归一化操作，训练集测试集的划分


In [ ]:
import torch
import torch.nn as nn

x_train = torch.from_numpy(x_train).type(torch.Tensor)
#将numpy类型转换为tensor类型
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
input_dim=1
#输入数据的特征维度。表示输入数据每个样本有一个特征

hidden_dim=32
#隐藏层的维度，模型中隐藏层的神经元数量

num_layers=2
#模型中的层数。这里表示包含两个隐藏层

output_dim=1
#输出数据的维度

num_epochs=100
#训练的轮数，表示对训练集进行100次训练
#并不是训练轮数越多越好，注意过拟合的问题




In [ ]:
#定义一个LSTM模型，完全不会写，只会传参数
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
#创建了一个LSTM模型。使用给定的超参数input_dim、hidden_dim、output_dim和num_layers来初始化LSTM模型的输入维度、隐藏层维度、输出维度和层数。
criterion = torch.nn.MSELoss()
#定义了一个均方误差损失函数（MSELoss）。这个损失函数用于衡量模型输出与目标输出之间的差异。
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

#定义了一个Adam优化器。该优化器用于更新模型的参数，以最小化损失函数。model.parameters()获取了LSTM模型的可学习参数，而lr=0.01表示学习率为0.01。